### 层和块

- 神经网络研究人员已经从考虑单个人工神经元的行为转变为从**层**的角度构思网络，通常在设计架构时考虑的是更粗粒度的**块（block）**

- 事实证明，研究讨论“比单个层大”但"比整个模型小"的组件更有价值。例如，在计算机视觉中广泛流行的 ResNet-152 架构就有数百层，**这些层是由层组（group of layers）的重复模式组成**

- **神经网络块的概念：块（block）可以描述单个层、由多个层组成的组件，或整个模型本身**

- 从编程的角度来看，**块由类（class）表示，我们只需要考虑前向传播函数和必要的参数即可。**

- **`nn.Sequential`定义了一种特殊的`Module`，即在Pytorch中表示一个块的类，它维护一个由`Module`组成的有序列表**

- **`net(X)`实际上是`net.__call__(X)`的简写**

- **块（block）必须提供的基本功能**：
  1. **将输入数据作为其前向传播函数的参数**
  2. 通过前向传播函数来生成输出
  3. **计算其输出关于输入的梯度，可通过反向传播函数进行访问。通常这是自动完成的**
  4. **存储和访问**前向传播计算所需的参数
  5. **根据需要初始化模型参数**

### 小结
- **一个块可以由许多层组成；一个块可以由许多块组成**
- 块可以包含代码
- 块负责大量的内部处理，**包括参数初始化和反向传播**
- 层和块的顺序连接由`Sequential`块处理

首先，我们回顾一下**多层感知机**

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0242,  0.1153,  0.2040,  0.0287, -0.0266,  0.0159,  0.0609,  0.0128,
          0.2768,  0.1767],
        [-0.0182,  0.0165,  0.1345,  0.0110, -0.0769, -0.0263,  0.0359,  0.0306,
          0.1647,  0.1684]], grad_fn=<AddmmBackward0>)

### 自定义块

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    # 定义模型的前向传播，即如何根据输入X计算返回所需要的模型输出
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

**实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层**

In [6]:
net = MLP()
net(X)

tensor([[-0.0072, -0.2892,  0.1307, -0.0346,  0.0774,  0.3824,  0.0403,  0.0860,
          0.1285,  0.0466],
        [-0.0055, -0.2632,  0.1058, -0.0091,  0.0549,  0.1241, -0.0058, -0.1128,
          0.1322,  0.0235]], grad_fn=<AddmmBackward0>)

### 顺序块

- 为了构建我们自己的简化的`MySequiential`，只需要定义下面**两个关键函数**：

  1. **将块逐个追加到列表中的函数**
  2. **前向传播函数**，用于将输入按追加块的顺序传递给块组成的“链条”

> **`MySequential`的用法与`Sequential`一样**

In [10]:
class MySequential(nn.Module):
    # 提供了与Sequential类相同的功能
    # def __init__(self, *args):
        # super().__init__()
        # for idx, module in enumerate(args):
            # # 这里，`module`是`Module`子类的一个实例。我们把它保存在'Module'类的成员变量
            # # 变量`_modules` 中。`_modules` 的类型是 OrderedDict。
            # self._modules[str(idx)] = module
    
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block    # 这里_modules是OrderedDict类型
            # `_modules`的主要优点是：在模块的参数初始化过程中，系统知道在`_modules`字典中查找相应的模块。
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1484, -0.0229,  0.0901, -0.0608,  0.0258,  0.0669, -0.1206, -0.0817,
         -0.1542,  0.1571],
        [ 0.0770,  0.0306,  0.0601, -0.0318,  0.1206,  0.0801, -0.1502, -0.0202,
         -0.0664,  0.1646]], grad_fn=<AddmmBackward0>)

**在正向传播函数中执行代码，有时`torch.Sequential`无法满足需求，需要我们自定义函数**

In [12]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)    # 不可训练参数
        self.linear = nn.Linear(20, 20)
    
    def forward(self, x):
        x = self.linear(x)
        x = F.relu(torch.mm(x, self.rand_weight) + 1)
        x = self.linear(x)
        while x.abs().sum() > 1:
            x /= 2
        return x.sum()  # 返回标量

net = FixedHiddenMLP()
net(X)

tensor(0.2683, grad_fn=<SumBackward0>)

**组合搭配各种组合块方法**

In [21]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.3256, grad_fn=<SumBackward0>)